# Preprocess

> Functions to preprocess the data

In [1]:
#| default_exp preprocess

In [2]:
#| hide
from nbdev.showdoc import *
import sys
sys.path.append("..")
from OPSIDIAN_nbdev.cfd_utils import clean_sys_path
clean_sys_path()

Cleaned sys.path:
c:\Users\ASUS\anaconda3\envs\opsidian
c:\Users\ASUS\anaconda3\envs\opsidian\lib\site-packages
C:\Program Files\FreeCAD 0.21\bin
C:\Users\ASUS\AppData\Roaming\FreeCAD\Mod\CfdOF
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\win32
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\win32\lib
C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\Pythonwin
Cleaned sys.path:
c:\Users\ASUS\anaconda3\envs\opsidian
c:\Users\ASUS\anaconda3\envs\opsidian\lib\site-packages
C:\Program Files\FreeCAD 0.21\bin
C:\Users\ASUS\AppData\Roaming\FreeCAD\Mod\CfdOF
C:\Users\ASUS\AppData\Roaming\FreeCAD\Mod\CfdOF\.\
C:\Program Files\FreeCAD 0.21\Mod\Web
C:\Program Files\FreeCAD 0.21\Mod\Tux
C:\Program Files\FreeCAD 0.21\Mod\Test
C:\Program Files\FreeCAD 0.21\Mod\TechDraw
C:\Program Files\FreeCAD 0.21\Mod\Surface
C:\Program Files\FreeCAD 0.21\Mod\Start
C:\Program Files\FreeCAD 0.21\Mod\Spreadsheet
C:\Progra

In [3]:
#| export
import csv
import FreeCAD as App
import Part
import math
from pathlib import Path
from OPSIDIAN_nbdev.utils import gradient_to_spline_angle

In [4]:
#| export
centres = [(0, 0, 0), #plane 1
            (5.333, 0, -144.623), #plane 2
            (36.791, 0, -289.331), #plane 3
            (91.665, 0, -426.512), #plane 4
            (167.629, 0, -554.016), #plane 5
            (279.67, 0, -656.591), #plane 6
            (407.779, 0, -737.83), #plane 7
            (536.748, 0, -799.26), #plane 8
            (672.528, 0, -835.133), #plane 9
            (814.82, 0, -847.56), #plane 10
            (977.817, 0, -845.565), #plane 11
            (1138.74, 0, -831.448), #plane 12
            (1300.874, 0, -802.406), #plane 13
            (1458.53, 0, -761.29), #plane 14
            (1780.247, 0, -673.92), #plane 15
            (2101.965, 0, -586.55), #plane 16
            (2423.682, 0, -499.18), #plane 17
            (2745.40, 0, -411.81), #plane 18
            (3235.68, 0, -274.137), #plane 19
            (3560.09, 0, -178.196), #plane 20
            (3876.111, 0, -76.161), #plane 21
            (4049.151, 0, -25.67), #plane 22
            (4137.633, 0, -4.381), #plane 23
            (4223.608, 0, 7.547), #plane 24
            (4281.578, 0, 12.828), #plane 25
            (4375.87, 0, 17.209)] #plane 26

In [5]:
#| export
def get_spline_details()->list: # List containing the spline details.
    "This function gets the spline details and calculates the angle of the spline at the plane in radians and degrees if the columns are not already present."
    spline_details = []
    path = '../data/spline/spline_details.csv'
       
    with open(Path(path), mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            spline_details.append(row)

    for row in spline_details:
        if 'angle_rad' not in row:
            tangent_grad = float(row['tangent_grad'])
            angle_rad = gradient_to_spline_angle(tangent_grad)
            row['angle_rad'] = angle_rad
            row['angle_deg'] = math.degrees(angle_rad)

    return spline_details

In [6]:
show_doc(get_spline_details)

---

### get_spline_details

>      get_spline_details ()

*This function gets the spline details and calculates the angle of the spline at the plane in radians and degrees if the columns are not already present.*

In [7]:
#| export
def get_cartesian_coords(centre_coords:tuple, # Tuple containing the centre coordinates.
                         phi_rad:list, # List of phi angles in radians.
                         polar_radial_distance:list, # List of polar radial distances.
                         theta:float, # Angle of the spline at the plane in radians.
                         )->list: # List containing the cartesian coordinates.
    "This function converts the local polar coordinates to cartesian coordinates."
    cartesian_coords = []
    for i in range(len(phi_rad)):
        x_global = centre_coords[0] + polar_radial_distance[i] * math.cos(phi_rad[i]) * math.cos(theta) * 10
        y_global = centre_coords[1] + polar_radial_distance[i] * math.sin(phi_rad[i]) * 10
        z_global = centre_coords[2] + polar_radial_distance[i] * math.cos(phi_rad[i]) * math.sin(theta) * 10
        cartesian_coords.append((x_global, y_global, z_global))
    return cartesian_coords

In [8]:
show_doc(get_cartesian_coords)

---

### get_cartesian_coords

>      get_cartesian_coords (centre_coords:tuple, phi_rad:list,
>                            polar_radial_distance:list, theta:float)

*This function converts the local polar coordinates to cartesian coordinates.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| centre_coords | tuple | Tuple containing the centre coordinates. |
| phi_rad | list | List of phi angles in radians. |
| polar_radial_distance | list | List of polar radial distances. |
| theta | float | Angle of the spline at the plane in radians. |
| **Returns** | **list** | **List containing the cartesian coordinates.** |

In [9]:
#| export
def calculate_face_coords(baseline_factor:list, # List containing the baseline factors.
                          )->list: # List containing the coordinates of the faces of the tube.
    "This function calculates the coordinates of the faces of the tube using the baseline factors to determine the shape of the face."
    print('Calculating face coordinates...')
    df_spline = get_spline_details()
    faces_coordinates = []
    for plane in range(1, 27):
            phi_rad = []
            polar_radial_distance = []
            for baseline in range(1,3):
                path = f'../data/baseline_{baseline}/plane_{plane}/polar_coordinates.csv'
                with open(Path(path), mode='r') as file:
                    reader = csv.DictReader(file)
                    i = 0
                    for row in reader:
                        if baseline == 1:
                            polar_radial_distance.append(baseline_factor[baseline-1]*float(row['radial_distance']))
                            phi_rad.append(float(row['phi_rad']))
                        else:
                            polar_radial_distance[i] = (polar_radial_distance[i] + baseline_factor[baseline-1]*float(row['radial_distance']))
                            i += 1
            theta = float(next(row['angle_rad'] for row in df_spline if int(row['plane']) == plane))
            cartesian_coords = get_cartesian_coords(centres[plane-1], phi_rad, polar_radial_distance, theta)
            faces_coordinates.append(cartesian_coords)
    return faces_coordinates

In [10]:
show_doc(calculate_face_coords)

---

### calculate_face_coords

>      calculate_face_coords (baseline_factor:list)

*This function calculates the coordinates of the faces of the tube using the baseline factors to determine the shape of the face.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| baseline_factor | list | List containing the baseline factors. |
| **Returns** | **list** | **List containing the coordinates of the faces of the tube.** |

In [11]:
#| export
def generate_faces(faces_coordinate: list, # List containing the coordinates of the faces of the tube.
                   )->list: # List containing the shapes of the faces.
    "This function generates faces in free cad using the calculated face coordinates."
    face_shapes = []
    for face_coords in faces_coordinate:
        # create freeCAD points
        vec_points = [App.Vector(p[0], p[1], p[2]) for p in face_coords[1:]]
        vec_points.append(vec_points[0])

        # create spline from points
        spline = Part.BSplineCurve()
        spline.interpolate(vec_points)

        # Add spline to the app
        obj = App.ActiveDocument.addObject('Part::Feature', 'Spline')
        obj.Shape = spline.toShape()

        # recompute the document
        App.ActiveDocument.recompute()
        face_shapes.append(obj)
    return face_shapes

In [12]:
show_doc(generate_faces)

---

### generate_faces

>      generate_faces (faces_coordinate:list)

*This function generates faces in free cad using the calculated face coordinates.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| faces_coordinate | list | List containing the coordinates of the faces of the tube. |
| **Returns** | **list** | **List containing the shapes of the faces.** |

In [13]:
#| export
def add_spine()->Part.Shape: # Shape of the spine.
    "This function adds the spine to the freeCAD document."
    # create freeCAD points
    spine_points = [App.Vector(coord[0], coord[1], coord[2]) for coord in centres]

    # create spline from points
    spine = Part.BSplineCurve()
    spine.interpolate(spine_points)

    # Add spline to the app
    obj = App.ActiveDocument.addObject('Part::Feature', 'Spine')
    obj.Shape = spine.toShape()

    # recompute the document
    App.ActiveDocument.recompute()

    return obj

In [14]:
show_doc(add_spine)

---

### add_spine

>      add_spine ()

*This function adds the spine to the freeCAD document.*

In [15]:
#| export
def sweep(face_shapes: list, # List containing the shapes of the faces.
          spine: Part.Shape, # Shape of the spine.
          )->Part.Shape: # Shape of the sweep.
    "This function performs a sweep operation in freeCAD."
    # create sweep object
    sweep = App.ActiveDocument.addObject('Part::Sweep', 'Sweep')
    sweep.Sections = face_shapes
    sweep.Spine = (spine,['Edge1',])
    sweep.Solid = True
    sweep.Frenet = False

    # recompute the document
    App.ActiveDocument.recompute()

    return sweep

In [16]:
show_doc(sweep)

---

### sweep

>      sweep (face_shapes:list, spine:Part.Shape)

*This function performs a sweep operation in freeCAD.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| face_shapes | list | List containing the shapes of the faces. |
| spine | Shape | Shape of the spine. |
| **Returns** | **Shape** | **Shape of the sweep.** |

In [17]:
#| hide
import nbdev; nbdev.nbdev_export()